In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('AllInOneCustom.xlsx')

In [7]:
df['CGM_Data'] = df['CGM (mg / dl)'].fillna(df['CGM '])

In [8]:
df.drop(['CGM (mg / dl)', 'CGM '], axis=1, inplace=True)

In [9]:
df.to_excel('AllInOneCustom.xlsx', index=False)

In [5]:
df["Date"] = pd.to_datetime(df["Date"])

In [3]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %I:%M:%S %p')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour
df['Minute'] = df['Date'].dt.minute
df['Second'] = df['Date'].dt.second
df['AM/PM'] = df['Date'].dt.strftime('%p')

In [4]:
df.to_excel('AllInOneCustom.xlsx', index=False)

In [32]:
import numpy as np

### Create avgCGM Column and Calculate

- avgCGM is the average cgm values after the glucose level fluctuation
- After food intake CGM value fluctuation peaks between 1-2 hours
- Therefore the avgCGM values between 1-2 hours after the meal is taken

In [25]:
df['avgCGM'] = ''

In [26]:
# iterate each row
for index, row in df.iterrows():
    dietary_intake = row['Dietary intake']
    # if user input food
    if pd.isnull(dietary_intake):
        df.at[index, 'avgCGM'] = ''
    else:
        # Initialize
        avgCGM = np.nan
        # if dataframe is not over
        if index + 8 < len(df):
            avgCGM = df.iloc[index+4:index+8]['CGM_Data'].astype(float).mean()

        # store at index(to predict)
        df.at[index, 'avgCGM'] = avgCGM



In [27]:
df.to_excel('AllInOneCustomAvg.xlsx', index=False)

### Remove all the rows that don't have the avgCGM value and save to file

In [4]:

df.dropna(subset=['avgCGM'], inplace=True)

In [5]:
df.to_excel('AllInOneCustomAvg.xlsx', index=False)

### Remove rows where avgCGM is taken from the next person

In [3]:
for index, row in df.iterrows():
    dietary_intake = row['Dietary intake']
    # if user input food
    if pd.notnull(dietary_intake):
        if index + 4 < len(df):
            #get the file names from index to index+8
            file_names = df.iloc[index:index+8]['File Name']
            #checks if file names are unique
            if file_names.nunique() == 1:
                pass
            else:
                df.drop(index,inplace=True)
        
df.to_excel('AllInOneCustomAvg.xlsx', index=False)

### Removing rows where Dietary intake has the value "data not available"

In [6]:
indices_to_remove = []

for index, row in df.iterrows():
    if row['Dietary intake'] == 'data not available':
        indices_to_remove.append(index)

df = df.drop(indices_to_remove)

In [7]:
df.to_excel('AllInOneCustomAvg.xlsx', index=False)

# Converting a json array with Foods and their GI values into a excel file

In [15]:
import json
from openpyxl import Workbook

In [19]:

# Open the JSON file and load the data
with open('data.json', 'r') as file:
    data = json.load(file)
    for i in range(5):
        if len(data) > i:
            print(data[i])

KeyError: 0